In [2]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
#from k_class_perceptron import k_class_perceptron
from sklearn.model_selection import train_test_split, KFold
from prettytable import PrettyTable
from muti_class_perceptron_1vRest import k_class_perceptron_1vRest
from multi_class_perceptron_1v1 import k_class_perceptron_1v1
#from skmultilearn.adapt import MLkNN
#from kernel_Perceptron_1 import kernel_Perceptron

data = pd.read_table("zipcombo.dat", sep="\s+")
#data = pd.read_table("dtrain123.dat", sep="\s+")
data = np.array(data)

In [3]:
def k_fold_crossvalidation(k, data_x, data_y, d, iteration, kernel = "poly"):
	#split data into k segments averagely
	kf=KFold(n_splits=k)
	a = kf.split(data_x)
	mistake_list = []
	for train_data_index, test_data_index in a:
		#split data set
		x_train = data_x[train_data_index]
		x_test = data_x[test_data_index]
		y_train = data_y[train_data_index]
		y_test = data_y[test_data_index]
		#perceptron
		kp = k_class_perceptron_1vRest(len(x_train),x_train,y_train,10,d, kernel)
		kp.train(iteration)
		mistakes = test_classifiers(x_test,y_test,kp)
		mistake_list.append(mistakes)
		print("-------------------")
	sum = 0
	for i in range(len(mistake_list)):
		sum+= mistake_list[i]
	mean_mistake = sum/len(mistake_list)
	return mean_mistake

In [ ]:
def test_classifiers(test_data,test_labels,perceptron):
	'''
	mistake_count = 0
	for i in range(len(test_data)):
		confidence, result = perceptron.predict(test_data[i])
		if(result != test_labels[i]):
			mistake_count = mistake_count + 1
	return mistake_count
	'''
	mistake_count = 0
	confident_mtx = perceptron.predict_mtx(test_data)
	mistake_count = 0
	for i in range(len(confident_mtx)):
		indivadual_confident_list = confident_mtx[i].tolist()
		predict_label = indivadual_confident_list.index(max(indivadual_confident_list))
		if(int(predict_label) != int(test_labels[i])):
			mistake_count += 1
	return mistake_count

In [4]:
def part1_2():
	interation = 5
	result_mtx = np.zeros(shape=(20,2))
	for i in range(20):
		data_train ,data_test = train_test_split(data,test_size=0.2)
		x_train = data_train[:,1:]
		y_train = data_train[:,0]
		x_test = data_test[:,1:]
		y_test = data_test[:,0]
		mistake_list = []
		for d in range(1,8):
			print("-----------------------")
			print("now d is: "+ str(d))
			print("-----------------------")
			mistake = k_fold_crossvalidation(5, x_train, y_train, d, interation)
			mistake_list.append(mistake)
		d_star = mistake_list.index(min(mistake_list))+1
		kp = k_class_perceptron_1vRest(len(x_train),x_train,y_train,10,d_star)
		kp.train(interation)
		test_error = test_classifiers(x_test,y_test,kp)
		result_mtx[i][0] = d_star
		result_mtx[i][1] = test_error
	mean_error = np.sum(result_mtx, axis = 0)[1]/20
	error_std = np.std(result_mtx, axis = 0)[1]
	mean_d_star =  np.sum(result_mtx, axis = 0)[0]/20
	d_star_std = np.std(result_mtx, axis = 0)[0]
	print("-------------")
	print("mean test error: "+ str(mean_error) + " ± " + str(error_std))
	print("mean d*: " + str(mean_d_star) + " ± " + str(d_star_std))

In [ ]:
part1_2()